# Exploratory Data Analysis of Epicurious Scrape in a JSON file

This is an idealized workflow for Aaron Chen in looking at data science problems. It likely isn't the best path, nor has he rigidly applied or stuck to this ideal, but he wishes that he worked this way more frequently.

## Purpose: Work through some exploratory data analysis of the Epicurious scrape on stream. Try to write some functions to help process the data.

### Author: Aaron Chen


---

### If needed, run shell commands here

In [ ]:
# !python -m spacy download en_core_web_sm

---

## External Resources

List out references or documentation that has helped you with this notebook

### Code
Regex Checker: https://regex101.com/

#### Scikit-learn
1. https://scikit-learn.org/stable/modules/decomposition.html#latent-dirichlet-allocation-lda
2. 

### Data

For this notebook, the data is stored in the repo base folder/data/raw

### Process

Are there steps or tutorials you are following? Those are things I try to list in Process

___

## Import necessary libraries

In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
# import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.base import TransformerMixin
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import spacy
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
# from spacy.lemmatizer import Lemmatizer
from tqdm import tqdm
from typing import Any

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


---

## Define helper functions

My workflow is to try things with code cells, then when the code cells get messy and repetitive, to convert into helper functions that can be called.

When the helper functions are getting used a lot, it is usually better to convert them to scripts or classes that can be called/instantiated

In [2]:
def custom_lemmatizer(ingredients: list) -> Any: # spacy nlp.Doc
    """This takes in a string representing the recipe and an NLP model and lemmatize with the NER. 
    
    Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    Remove punctuation

    Args:
        ingredients: string
        nlp_mod: spacy model (try built in first, by default called nlp)
    
    Returns:
        nlp.Doc
    """
    lemmas = [token.lemma_ for token in ingredients if (token.is_alpha and token.pos_ not in ["PRON", "VERB"] and len(token.lemma_) > 1)]
    return lemmas
    # return doc

In [3]:
def custom_preprocessor(recipe_ingreds: str) -> list:
    """This function replaces the default sklearn CountVectorizer preprocessor to use spaCy. sklearn CountVectorizer's preprocessor only performs accent removal and lowercasing.

    Args:
        A string to tokenize from a recipe representing the ingredients used in the recipe

    Returns:
        A list of strings that have been de-accented and lowercased to be used in tokenization
    """
    preprocessed = [token for token in nlp(recipe_ingreds)]

    return preprocessed

In [4]:
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[: -n_top_words - 1 : -1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.invert_yaxis()
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

### Import local script

I started grouping this in with importing libraries, but putting them at the bottom of the list

In [5]:
import project_path

import src.dataframe_preprocessor as dfpp

pyLDAvis.enable_notebook()

---

## Define global variables 
### Remember to refactor these out, not ideal

In [6]:
data_path = "../../data/recipes-en-201706/epicurious-recipes_m2.json"
food_stopwords_path = "../../food_stopwords.csv"

---

## Running Commentary

1. I used numbered lists to keep track of things I noticed

### To Do

1. Try to determine consistency of nested data structures
   1. Is the photoData or number of things inside photoData the same from record to record
   2. What about for tag?

Data wasn't fully consistent but logic in helper function helped handle nulls

2. How to handle nulls?
   1. Author      Filled in with "Missing Author"
   2. Tag         Filled in with "Missing Cuisine"
3. ~~Convert pubDate to actual timestamp~~  
4. ~~Convert ScrapeDate to actual timestamp~~
   1. This was ignored as the datestamp was not useful (generally within minutes of the origin of UNIX time)
   
**5. Append new columns for relevant nested structures and unfold them**

6. Determine actual types of `ingredients` and `prepSteps`
7. Continue working through test example of single recipe to feed into spaCy and then sklearn.feature_extraction.text stack
8. Will need to remove numbers, punctuation

---

## Importing and viewing the data as a dataframe

In [7]:
repo = pd.read_json(path_or_buf=data_path) # type:ignore
pd.read_json(data_path, typ='frame') # type:ignore

dfpp.preprocess_dataframe(df=repo) # type:ignore
print(repo.shape)
repo.head(10) # type:ignore

recipe_megalist = [ingred for recipe in repo['ingredients'].tolist() for ingred in recipe]


nlp = spacy.load("en_core_web_sm")

# this is a redeem for variable naming mixed with a free pun-ish me daddy, flushtrated will be the list of all stopword to exclude so named because we're throwing these words down the drain

flushtrated = {x for x in pd.read_csv(food_stopwords_path)}
flushtrated = flushtrated.union(STOP_WORDS)
flushtrated_list = list(flushtrated)

(34656, 14)


/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/spacy/util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
cv = CountVectorizer(strip_accents='unicode', lowercase=True, preprocessor=custom_preprocessor, tokenizer=custom_lemmatizer, stop_words=flushtrated_list, ngram_range=(1,4), min_df=10)
repo_transformed = cv.fit_transform(tqdm(recipe_megalist))
cv.get_feature_names_out().shape

In [ ]:
repo_transformed.shape

We can try to filter out the adjectives in the lemmatization step, because spaCy allows filtering based on Parts of Speech. But this might exclude them from the ngrams. Let's try augmenting stopwords and excluding colors that way.

In [ ]:
lda_20 = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
lda_20_repo_transformed = lda_20.fit_transform(repo_transformed)

In [ ]:
pyLDAvis.sklearn.prepare(lda_20, repo_transformed, cv)

## Manual Topic Labeling Based on LDA
1. aliums, alium prep, chocolate and rosemary
2. oil, cheese, game meats
3. peppers and parsley

Based on these three topics, I think it is better to train a new model, since these models don't seem to carry much information

In [ ]:
cv_auto_stopwords_085 = CountVectorizer(strip_accents='unicode', lowercase=True, preprocessor=custom_preprocessor, tokenizer=custom_lemmatizer, stop_words=None, ngram_range=(1,4), max_df=0.85)

repo_transformed_auto_stopwords_085 = cv_auto_stopwords_085.fit_transform(tqdm(recipe_megalist))
cv_auto_stopwords_085.get_feature_names_out().shape
lda_20_auto_stopwords_085 = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
lda_20_repo_transformed_auto_stopwords_085 = lda_20_auto_stopwords_085.fit_transform(repo_transformed_auto_stopwords_085)
pyLDAvis.sklearn.prepare(lda_20_auto_stopwords_085, repo_transformed_auto_stopwords_085, cv_auto_stopwords_085)

This LDA is probably less useful

1. Has a lot of prep and units of measurement
2. 

In [8]:
additional_to_exclude = {'red', 'green', 'black', 'yellow', 'white', 'inch', 'mince', 'chop', 'fry', 'trim', 'flat', 'beat', 'brown', 'golden', 'balsamic', 'halve', 'blue', 'divide', 'trim', 'unbleache', 'granulate'}
flushtrated_augment = flushtrated.union(additional_to_exclude)
flushtrated_augment = list(flushtrated_augment)

cv_stopwords_aug = CountVectorizer(strip_accents='unicode', lowercase=True, preprocessor=custom_preprocessor, tokenizer=custom_lemmatizer, stop_words=flushtrated_augment, ngram_range=(1,4), min_df=10)

repo_transformed_stopwords_aug = cv_stopwords_aug.fit_transform(tqdm(recipe_megalist))
cv_stopwords_aug.get_feature_names_out().shape
lda_20_stopwords_aug = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
lda_20_repo_transformed_aug = lda_20_stopwords_aug.fit_transform(repo_transformed_stopwords_aug)
pyLDAvis.sklearn.prepare(lda_20_stopwords_aug, repo_transformed_stopwords_aug, cv_stopwords_aug)

  0%|                                                                       | 0/341271 [00:00<?, ?it/s]/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['Frank', 'alternative', 'american', 'annie', 'asian', 'balance', 'band', 'barrel', 'bay', 'bayou', 'beam', 'beard', 'bell', 'betty', 'bird', 'blast', 'bob', 'bone', 'breyers', 'calore', 'carb', 'card', 'chachere', 'change', 'circle', 'coffee', 'coil', 'country', 'cow', 'crack', 'cracker', 'crocker', 'crystal', 'dean', 'degree', 'deluxe', 'direction', 'duncan', 'earth', 'eggland', 'ener', 'envelope', 'eye', 'fantastic', 'far', 'fat', 'feather', 'flake', 'foot', 'fourth', 'frank', 'french', 'fusion', 'genoa', 'genovese', 'germain', 'giada', 'gold', 'granule', 'greek', 'hamburger', 'helper', 'herbe', 'hines', 'hodgson', 'hunt', 'instruction', 'interval', 'italians

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    2.4s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    2.4s remaining:   26.3s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    2.4s remaining:   17.0s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    2.5s remaining:   12.3s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    2.5s remaining:    9.5s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    2.5s remaining:    7.5s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    2.5s remaining:    6.1s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    2.5s remaining:    5.0s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    2.5s remaining:    4.2s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    2.5s remaining:    3.6s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    2.6s remaining:    3.0s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    0.6s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.6s remaining:    6.8s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    0.6s remaining:    4.4s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    0.6s remaining:    3.1s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    0.6s remaining:    2.4s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    0.6s remaining:    1.9s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    0.6s remaining:    1.6s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    0.6s remaining:    1.3s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    0.6s remaining:    1.1s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    0.7s remaining:    0.9s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    0.7s remaining:    0.8s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapsed:    0.7s remaining:    0.7s
[Parallel(n_jobs=24)]: Done  13 out of  24 | el

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12    -0.268448  0.171293       1        1  7.263871
9     -0.242929  0.106905       2        1  6.513923
2      0.029426 -0.097528       3        1  6.156704
10     0.194096  0.107833       4        1  5.834851
17    -0.097421 -0.127486       5        1  5.833276
8      0.033426 -0.076291       6        1  5.704329
0     -0.025609 -0.118988       7        1  5.142124
18     0.216873  0.193254       8        1  5.084135
19     0.078221 -0.084846       9        1  5.021771
15     0.107487  0.136222      10        1  4.774017
3     -0.136144  0.050588      11        1  4.742466
4      0.021680 -0.109641      12        1  4.652382
14     0.062603 -0.005047      13        1  4.612657
1      0.066177 -0.092014      14        1  4.500687
11    -0.007409 -0.186646      15        1  4.498825
6      0.145913  0.207709      16        1  4.352312
13     0.066791 -0.084507      17        1  4.157788
16    -0.069622 -0.107790      18        1  4.015531
5      0.020940 -0.017916      19        1  3.588667
7     -0.196052  0.134895      20        1  3.549682, topic_info=          Term          Freq         Total Category  logprob  loglift
3320      salt  18690.000000  18690.000000  Default  30.0000  30.0000
3650     sugar  15203.000000  15203.000000  Default  29.0000  29.0000
1257    butter  13551.000000  13551.000000  Default  28.0000  28.0000
1922       egg  10468.000000  10468.000000  Default  27.0000  27.0000
3001    pepper  13483.000000  13483.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
1046     berry    303.404067    420.990499  Topic20  -4.3637   3.0108
1132   bottled    345.269050    630.637245  Topic20  -4.2345   2.7359
2781       oil   1489.765531  18896.236315  Topic20  -2.7724   0.7980
3152  preserve    151.810592    304.886156  Topic20  -5.0561   2.6410
2934     paste    195.174625   1531.323902  Topic20  -4.8049   1.2783

[867 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
13       10  0.993916         Angostura
15       10  0.991905  Angostura bitter
84        6  0.806466        California
84       12  0.006521        California
84       17  0.184770        California
...     ...       ...               ...
4032     11  0.109222              zest
4032     15  0.890435              zest
4033     15  0.994970        zest lemon
4039     15  0.988208       zest orange
4043      5  0.999032          zucchini

[1014 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 10, 3, 11, 18, 9, 1, 19, 20, 16, 4, 5, 15, 2, 12, 7, 14, 17, 6, 8])

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


These topic models/word groupings also don't seem to make much sense, so let's throw this into a TF-IDF and see what happens, even though the authors of LDA don't like doing this.

In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()

repo_tfidf_stopwords_aug = tfidf.fit_transform(repo_transformed_stopwords_aug)
tfidf_lda_20_stopwords_aug = LatentDirichletAllocation(n_components=20, n_jobs=-1, verbose=100, random_state=200)
tfidf_lda_20_repo_transformed_aug = tfidf_lda_20_stopwords_aug.fit_transform(repo_tfidf_stopwords_aug)
pyLDAvis.sklearn.prepare(tfidf_lda_20_stopwords_aug, repo_tfidf_stopwords_aug, cv_stopwords_aug)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    2.0s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    2.1s remaining:   22.7s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    2.1s remaining:   14.8s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    2.1s remaining:   10.7s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    2.1s remaining:    8.2s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    2.2s remaining:    6.5s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    2.2s remaining:    5.3s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    2.2s remaining:    4.4s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    2.2s remaining:    3.6s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    2.2s remaining:    3.1s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    2.2s remaining:    2.6s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapse

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[Parallel(n_jobs=24)]: Done   1 tasks      | elapsed:    0.5s
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.5s remaining:    6.0s
[Parallel(n_jobs=24)]: Done   3 out of  24 | elapsed:    0.6s remaining:    4.0s
[Parallel(n_jobs=24)]: Done   4 out of  24 | elapsed:    0.6s remaining:    2.8s
[Parallel(n_jobs=24)]: Done   5 out of  24 | elapsed:    0.6s remaining:    2.2s
[Parallel(n_jobs=24)]: Done   6 out of  24 | elapsed:    0.6s remaining:    1.7s
[Parallel(n_jobs=24)]: Done   7 out of  24 | elapsed:    0.6s remaining:    1.4s
[Parallel(n_jobs=24)]: Done   8 out of  24 | elapsed:    0.6s remaining:    1.2s
[Parallel(n_jobs=24)]: Done   9 out of  24 | elapsed:    0.6s remaining:    1.0s
[Parallel(n_jobs=24)]: Done  10 out of  24 | elapsed:    0.6s remaining:    0.8s
[Parallel(n_jobs=24)]: Done  11 out of  24 | elapsed:    0.6s remaining:    0.7s
[Parallel(n_jobs=24)]: Done  12 out of  24 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=24)]: Done  13 out of  24 | el

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12     0.202353  0.213939       1        1  6.128079
2     -0.017052 -0.040867       2        1  6.033571
9      0.167757  0.162223       3        1  5.834258
17     0.057734 -0.202707       4        1  5.804764
0      0.053824 -0.077143       5        1  5.602479
4     -0.025143 -0.102434       6        1  5.433710
6     -0.161813  0.013114       7        1  5.263957
10    -0.072701  0.104766       8        1  5.148689
3      0.119937  0.003307       9        1  4.943743
14     0.078602  0.112553      10        1  4.846614
13     0.022411 -0.216042      11        1  4.812587
8     -0.078211 -0.041697      12        1  4.788792
15    -0.057782  0.041690      13        1  4.755461
19     0.016599 -0.017181      14        1  4.677174
11     0.049828 -0.141037      15        1  4.653046
18    -0.258249  0.110564      16        1  4.543959
16     0.073278 -0.120564      17        1  4.490130
1     -0.301422  0.079102      18        1  4.201683
5      0.020868 -0.024654      19        1  4.086252
7      0.109183  0.143067      20        1  3.951055, topic_info=         Term          Freq         Total Category  logprob  loglift
3320     salt  16559.000000  16559.000000  Default  30.0000  30.0000
1257   butter  13029.000000  13029.000000  Default  29.0000  29.0000
3650    sugar  13910.000000  13910.000000  Default  28.0000  28.0000
1922      egg   9856.000000   9856.000000  Default  27.0000  27.0000
2818    onion   8290.000000   8290.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
3115     pork    455.521075    742.130782  Topic20  -3.6334   2.7431
1132  bottled    189.977493    327.040608  Topic20  -4.5080   2.6880
1524  chinese    153.904723    218.372971  Topic20  -4.7185   2.8813
2781      oil    344.121939   8836.402986  Topic20  -3.9139  -0.0145
3138   potato    159.990439   1891.651325  Topic20  -4.6797   0.7611

[943 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
13       13  0.993940         Angostura
15       13  0.990615  Angostura bitter
62        8  0.986928              Bosc
63        8  0.988043         Bosc pear
82       18  0.981040             Cajun
...     ...       ...               ...
4033     11  0.114934        zest lemon
4033     20  0.882905        zest lemon
4039     15  0.985236       zest orange
4043      3  0.998136          zucchini
4047     10  0.988998             árbol

[1344 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 3, 10, 18, 1, 5, 7, 11, 4, 15, 14, 9, 16, 20, 12, 19, 17, 2, 6, 8])

/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/awchen/Repos/Projects/MeaLeon/.venv/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


pyLDAvis calls to a deprecated function inside CountVectorizer, which is incompatible with TFIDF. Can we can find an alternate version?